In [8]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

model_name="BAAI/bge-large-en-v1.5"
model_kwargs = {"device": "cpu","trust_remote_code":True}
encode_kwargs = {"normalize_embeddings": True}


# text_splitter = RecursiveCharacterTextSplitter(chunk_size=250,
    #                                                    chunk_overlap=50)




embedding_function = SentenceTransformerEmbeddings(
model_name=model_name,
model_kwargs=model_kwargs,
encode_kwargs=encode_kwargs,
)



/home/hp/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
vectordb = Chroma(persist_directory="embed",embedding_function=embedding_function)
vectordb

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

def check_token_limit(new_documents):
  tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
  tokens=tokenizer(new_documents)
  n_tokens=len(tokens.input_ids)
  return n_tokens

In [11]:
k=100
def search(prompt):
  print("-----------")
  print(prompt)
  docs = vectordb.similarity_search_with_score(query=prompt,k=k)
  # print("Documents:")
  # print(len(docs))
  # print("::Documents")
  new_documents=" "
  for i in range(0,len(docs)):
    temp_doc=docs[i][0].page_content
    # print("Chunk : : \n : "+ temp_doc)
    check_doc=new_documents+temp_doc
    tokens=check_token_limit(check_doc)
    if tokens>4000:
      doc=temp_doc
      for x in range(0,5):
        doc= doc[:len(doc) // 2]
        temp_tokens=check_token_limit(doc+new_documents)
        if temp_tokens<=3950:
          new_documents =new_documents+doc + "\n\n--------------------\n\n"
          break


      return new_documents
    else:
      new_documents=check_doc + "\n\n-----------------\n\n"

  return new_documents

In [5]:
system_prompt="""You are a knowledgeable and professional assistant tasked with providing detailed information about construction contract. Your responses should be concise, informative, and strictly adhere to the terms and conditions outlined in the provided contract details."""

In [6]:
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage


# llm = Ollama(model="llama3:8b-text-q4_K_M")
 
llm = ChatOllama(
    model="llama3:8b-instruct-q4_K_M",
    n_ctx=8000,
    temperature=0.0,
    top_p=0.7,
    
    system_message=SystemMessage(content=system_prompt)
)


In [ ]:
llm.invoke("hello")

In [13]:
prompts = [
    "What is CONTRACTOR limit of liability and the liability cap?",
    "Under what condition COMPANY can terminate the contract?",
    "Can CONTRACTOR terminate the contract?",
    "Can CONTRACTOR claim extension of time and additional cost on account of force majeure?",
    "Is increase in quantity a COMPANY risk event or CONTRACTOR risk event?",
]


In [28]:
check_token_limit(new_documents)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


4447

In [11]:
import time 

time.sleep(2000)

NameError: name 'sleep' is not defined

In [12]:
new_documents=search(prompt[0])
question = " <Data>: \n" + new_documents + "\n\n ------------------- "+" \n\n <Question>: "+prompt[0]
print(question)

-----------
What is CONTRACTOR limit of liability and the liability cap?


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make s

 <Data>: 
 41.1 In the event any CONTRACTOR PERSONNEL are performing WORKS in the UAE, without limitation to its obligations and responsibilities, the CONTRACTOR shall obtain and maintain Workmen’s Compensation Insurance (Employer’s Liability) with a limit of liability of not less than USD 1,000,000 per occurrence.
41.2 In the event any CONTRACTOR PERSONNEL are performing WORKS outside the UAE, without limitation to its obligations and responsibilities, the CONTRACTOR shall obtain and maintain Workmen’s Compensation Insurance (Employer’s Liability) or insurance coverage of a comparable 
nature in accordance with the applicable LAW and/or the contracts of employment of the CONTRACTOR PERSONNEL.

-----------------

41.5 The CONTRACTOR shall obtain and maintain: Third Party Liability Insurance with a limit of not less than USD 10,000,000 per occurrence; and Pollution Liability Insurance to cover the risk arising from the performance of WORKS and any pollution emanating/originating from th

In [24]:
new_documents

' 41.1 In the event any CONTRACTOR PERSONNEL are performing WORKS in the UAE, without limitation to its obligations and responsibilities, the CONTRACTOR shall obtain and maintain Workmen’s Compensation Insurance (Employer’s Liability) with a limit of liability of not less than USD 1,000,000 per occurrence.\n41.2 In the event any CONTRACTOR PERSONNEL are performing WORKS outside the UAE, without limitation to its obligations and responsibilities, the CONTRACTOR shall obtain and maintain Workmen’s Compensation Insurance (Employer’s Liability) or insurance coverage of a comparable \nnature in accordance with the applicable LAW and/or the contracts of employment of the CONTRACTOR PERSONNEL.\n\n\n-----------------\n\n\n41.5 The CONTRACTOR shall obtain and maintain: Third Party Liability Insurance with a limit of not less than USD 10,000,000 per occurrence; and Pollution Liability Insurance to cover the risk arising from the performance of WORKS and any pollution emanating/originating from t

In [12]:
question

' <Data>: \n \n \n 41.1 In the event any CONTRACTOR PERSONNEL are performing WORKS in the UAE, without limitation to its obligations and responsibilities, the CONTRACTOR shall obtain and maintain Workmen’s Compensation Insurance (Employer’s Liability) with a limit of liability of not less than USD 1,000,000 per occurrence.\n41.2 In the event any CONTRACTOR PERSONNEL are performing WORKS outside the UAE, without limitation to its obligations and responsibilities, the CONTRACTOR shall obtain and maintain Workmen’s Compensation Insurance (Employer’s Liability) or insurance coverage of a comparable \nnature in accordance with the applicable LAW and/or the contracts of employment of the CONTRACTOR PERSONNEL.\n\n\n-----------------\n\n\n41.5 The CONTRACTOR shall obtain and maintain: Third Party Liability Insurance with a limit of not less than USD 10,000,000 per occurrence; and Pollution Liability Insurance to cover the risk arising from the performance of WORKS and any pollution emanating/o

In [13]:
llm.invoke(question)

AIMessage(content="Based on the provided text, it appears that there are several instances where the CONTRACTOR's liability is limited or capped. Here are some specific examples:\n\n1. Article 42.3: The CONTRACTOR shall indemnify the COMPANY INDEMNIFIED PARTIES up to a maximum of USD 1,000,000 per occurrence for losses arising out of or in connection with any claim by any PERSON against the COMPANY INDEMNIFIED PARTIES.\n2. Article 42.12 (ii): The COMPANY shall indemnify the CONTRACTOR INDEMNIFIED PARTIES up to a maximum of USD 1,000,000 per occurrence for losses arising out of or in connection with any claim by any PERSON against the CONTRACTOR INDEMNIFIED PARTIES.\n\nIt appears that the liability cap is USD 1,000,000 per occurrence. However, it's worth noting that there may be other instances where the liability is limited or capped, and these examples are not exhaustive.", response_metadata={'model': 'llama3:8b-instruct-q4_K_M', 'created_at': '2024-06-30T23:17:40.174711091Z', 'messag

In [21]:
new_documents=search(prompt[0])
question = "Carefully read this text to answer below question : " + new_documents + "    \n\n ------------------- "+prompt[0]
llm.invoke(question)


-----------
What is CONTRACTOR limit of liability and the liability cap?


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make s

AIMessage(content='Based on the provided text, there is no specific mention of a "CONTRACTOR limit of liability" or a "liability cap". However, it can be inferred that the CONTRACTOR\'s liability is limited in certain situations:\n\n1. Article 33.2(a) states that the CONTRACTOR shall rectify any loss or damage to the WORKS prior to issuance of the PROVISIONAL ACCEPTANCE CERTIFICATE at its sole cost and expense.\n2. Article 42.4 Pollution from CONTRACTOR PROPERTY mentions that the CONTRACTOR shall indemnify and hold harmless the COMPANY INDEMNIFIED PARTIES from and against all losses, claims, costs, liabilities, damages, and expenses arising from or in connection with any pollution originating from the CONTRACTOR\'s performance of the WORKS and/or the AGREEMENT.\n3. Article 43.1 states that the CONTRACTOR shall not assign, novate, or otherwise transfer any of its rights, obligations, or interests under and in connection with the AGREEMENT without the prior written consent of the COMPANY

In [13]:
new_documents=search(prompt[2])
question = "Carefully read this text to answer below question : " + new_documents + "    \n\n ------------------- "+prompt[2]
llm.invoke(question)  

-----------
Can CONTRACTOR terminate the contract?


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make s

AIMessage(content='Based on the text, it appears that the CONTRACTOR may not be able to terminate the contract unilaterally. Article 35.3(a)(v) mentions that the COMPANY shall have the right to terminate the AGREEMENT with immediate effect if certain circumstances exist, including where the CONTRACTOR fails to meet any of the LAW regarding HSE or the HSE REQUIREMENTS.\n\nAdditionally, Article 20.4 states that the CONTRACTOR shall ensure that its SUBCONTRACTORS and VENDORS comply with the requirements set out in the AGREEMENT, and failure to do so may entitle the COMPANY to terminate the contract.\n\nFurthermore, Article 24.6(d) mentions that the COMPANY shall not be liable for any claim by the CONTRACTOR arising out of the elimination of part of the WORKS or the exercise of its rights under Article 24.6(c)(i), which suggests that the COMPANY has significant discretion in determining whether to terminate the contract.\n\nIn contrast, there is no provision in the text that explicitly gra

In [14]:
new_documents=search(prompt[3])
question = "Carefully read this text to answer below question : " + new_documents + "    \n\n ------------------- "+prompt[3]
llm.invoke(question)  

-----------
Can CONTRACTOR claim extension of time and additional cost on account of force majeure?


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make s

AIMessage(content='Based on the text, it appears that there is no specific provision that explicitly mentions "force majeure" as a reason for claiming an extension of time and additional costs. However, Article 29.1 does mention that the CONTRACTOR shall not be entitled to any adjustments to the AGREEMENT PRICE and/or the KEY MILESTONE DATES or other relief if the change in LAW is due to a default by the CONTRACTOR.\n\nArticle 25.5 also mentions CONCURRENCY as a reason for which the CONTRACTOR shall retain the right to claim an adjustment to the KEY MILESTONE DATES, but not entitled to any adjustment to the AGREEMENT PRICE.\n\nHowever, it can be argued that force majeure is implicitly covered under Article 34.1, which states that the COMPANY\'s instruction, decision or act shall not constitute a variation if made in order to ensure compliance with the CONTRACTOR\'s obligations under the agreement, subject to Article 29, LAW. This implies that the CONTRACTOR may be entitled to claim an 

In [11]:
new_documents=search(prompt[4])
question = "Carefully read this text to answer below question : " + new_documents + "    \n\n ------------------- "+prompt[4]
llm.invoke(question)  

-----------
Is increase in quantity a COMPANY risk event or CONTRACTOR risk event?


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make s

AIMessage(content='Based on the provided text, it appears that an increase in quantity is not mentioned as a "COMPANY risk event" or a "CONTRACTOR risk event".\n\nArticle 25.1 lists several events or circumstances that may cause delays to the WORKS, including:\n\n* (a) any breach of the AGREEMENT or act of prevention by the COMPANY, its AFFILIATES or their respective agents or employees;\n* (b) subject to Article 12.10, any delays caused by ASSOCIATED WORKS CONTRACTORS working at the SITE, except to the extent that the delays caused by ASSOCIATED WORKS CONTRACTORS are attributable to the CONTRACTOR\'s breach of Article 12;\n* (c) the circumstances referred to in Articles 4.3(c), 5.2(a), 7.9(e), 8.4, 11.10(b), 11.11(b), 13.4, 23.7(c), or 29.1;\n* (d) a suspension of the WORKS in the circumstances referred to in Article 34.1 or 34.3;\n\nThere is no mention of an increase in quantity as a risk event that may cause delays to the WORKS.\n\nHowever, it\'s worth noting that Article 7.9(b) req

In [1]:
print(question)

NameError: name 'question' is not defined

In [14]:
import requests
import json
for prompt in prompts:
 
 new_documents=search(prompt)
 question = " <Data>: \n" + new_documents + "\n\n ------------------- "+" \n\n <Question>: "+prompt
 
 url = "http://localhost:11434/api/generate"

 payload = {
    "model": "llama3:8b-instruct-q4_K_M",
    "options": {"num_ctx": 8000,"temperature":0.0,"top_p":0.5},
    "system":system_prompt,
    "prompt": question,
    "stream": False
 }

# Send the POST request
 response = requests.post(url, data=json.dumps(payload), headers={"Content-Type": "application/json"})

# Print the response

 if response.status_code == 200:
    print("Response:", response.json())
 else:
    print(f"Failed to get a response. Status code: {response.status_code}, Response: {response.text}")


-----------
What is CONTRACTOR limit of liability and the liability cap?


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make s

Response: {'model': 'llama3:8b-instruct-q4_K_M', 'created_at': '2024-07-01T20:56:32.261267721Z', 'response': "According to Article 41.5(a) and (b), the CONTRACTOR's limit of liability for Third Party Liability Insurance and Pollution Liability Insurance is USD 10,000,000 per occurrence.\n\nAdditionally, according to Article 42.10, the total liability of either PARTY to the other PARTY arising under or in connection with the AGREEMENT shall be limited to the total liability cap set out in the FORM OF AGREEMENT, which is not specified in this text.", 'done': True, 'context': [128006, 9125, 128007, 198, 198, 2675, 527, 264, 42066, 323, 6721, 18328, 51920, 449, 8405, 11944, 2038, 922, 8246, 5226, 13, 4718, 14847, 1288, 387, 64694, 11, 39319, 11, 323, 26549, 49553, 311, 279, 3878, 323, 4787, 33740, 304, 279, 3984, 5226, 3649, 13, 128009, 128006, 882, 128007, 271, 220, 45495, 27916, 720, 220, 3174, 13, 16, 763, 279, 1567, 904, 17564, 878, 57477, 82951, 527, 16785, 32904, 50, 304, 279, 47949,

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make s

Response: {'model': 'llama3:8b-instruct-q4_K_M', 'created_at': '2024-07-02T00:02:40.175725524Z', 'response': "According to the provided contract details, the COMPANY can terminate the contract under the following conditions:\n\n1. Article 35.1: The COMPANY may terminate the AGREEMENT with immediate effect by written notice to the CONTRACTOR if:\n\t* The CONTRACTOR has failed to achieve any of the PERFORMANCE GUARANTEES during the PERFORMANCE TESTS on three (3) occasions and Article 16.3(d) applies.\n\t* Any express limitations on the liability of the CONTRACTOR under the AGREEMENT, including the overall maximum amount for DELAY LIQUIDATED DAMAGES in relation to delay to PROVISIONAL ACCEPTANCE (but excluding any overall maximum amounts for other KEY MILESTONES), have been reached or exceeded.\n\t* ABANDONMENT by the CONTRACTOR.\n2. Article 35.2: The COMPANY may terminate the AGREEMENT with immediate effect by written notice to the CONTRACTOR if:\n\t* An INSOLVENCY EVENT occurs in relati

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make s

Response: {'model': 'llama3:8b-instruct-q4_K_M', 'created_at': '2024-07-02T01:54:30.187582227Z', 'response': "Based on the provided contract details, it appears that the CONTRACTOR may terminate the agreement in certain circumstances. Here are some specific instances where the CONTRACTOR may be able to terminate the contract:\n\n1. **Change of Control**: If there is a change of control of the CONTRACTOR or any person who directly or indirectly controls the CONTRACTOR (including an intermediate or ultimate parent company), the COMPANY may terminate the agreement with immediate effect by written notice to the CONTRACTOR (Article 35.5).\n2. **Suspension and Failure to Resume**: If the COMPANY suspends the WORKS and fails to resume performance, the CONTRACTOR may terminate the agreement by written notice to the COMPANY (Article 34.7(a)(ii)).\n3. **Non-Payment**: If the COMPANY fails to make payment of any properly invoiced and overdue undisputed amount, the CONTRACTOR may terminate the agr

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make s

Response: {'model': 'llama3:8b-instruct-q4_K_M', 'created_at': '2024-07-02T04:29:35.684737526Z', 'response': "According to Article 40, a FORCE MAJEURE EVENT is defined as an event or circumstance that satisfies the requirements of paragraphs (a) and (b) above. The following events or circumstances shall not be or be deemed to be a FORCE MAJEURE EVENT:\n\n* Unavailability of funds or non-payment of a sum of money by a PARTY or a THIRD PARTY\n* Financial hardship or inability to make a profit or achieve a satisfactory rate of return\n* SITE conditions, including extraordinary GROUND CONDITIONS\n\nIn Article 40.3, it is stated that during any period of the FORCE MAJEURE EVENT, the CONTRACTOR shall use its best efforts to safeguard and protect the WORKS and utilise the CONTRACTOR PERSONNEL in such a manner as to minimise the costs associated with the FORCE MAJEURE EVENT.\n\nArticle 34.6(c) states that if the WORKS are suspended in accordance with Article 34.3 (Suspension for a FORCE MAJEUR

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make s

In [ ]:
answer="According to Article 41.5(a), the CONTRACTOR's limit of liability for Third Party Liability Insurance is USD 10,000,000 per occurrence.\n2. **Liability cap**: The total liability cap is stated in Article 42.10 and is equal to 100% of the AGREEMENT PRICE.\n\nNote that the liability cap is a maximum amount that the COMPANY or the CONTRACTOR can be held liable for under the agreement, whereas the limit of liability refers to the specific amount insured against for each type of insurance (e.g., Third Party Liability Insurance)"
print(answer)

In [ ]:
check_token_limit(question)

In [ ]:
curl http://localhost:11434/api/generate  '{
  "model": "llama3:8b-instruct-q4_K_M",
  "options": {
    "num_ctx": 8000,
    "temperature": 0.0,
    "top_p": 0.5
  },
  "system": "You are a knowledgeable and professional assistant tasked with providing detailed information about insurance requirements for a construction contract. Your responses should be concise, informative, and strictly adhere to the terms and conditions outlined in the provided contract details.",
  "prompt": "<Data>: \n \n \n 41.1 In the event any CONTRACTOR PERSONNEL are performing WORKS in the UAE, without limitation to its obligations and responsibilities, the CONTRACTOR shall obtain and maintain Workmen’s Compensation Insurance (Employer’s Liability) with a limit of liability of not less than USD 1,000,000 per occurrence.\n41.2 In the event any CONTRACTOR PERSONNEL are performing WORKS outside the UAE, without limitation to its obligations and responsibilities, the CONTRACTOR shall obtain and maintain Workmen’s Compensation Insurance (Employer’s Liability) or insurance coverage of a comparable \nnature in accordance with the applicable LAW and/or the contracts of employment of the CONTRACTOR PERSONNEL.\n\n\n-----------------\n\n\n41.5 The CONTRACTOR shall obtain and maintain: Third Party Liability Insurance with a limit of not less than USD 10,000,000 per occurrence; and Pollution Liability Insurance to cover the risk arising from the performance of WORKS and any pollution emanating/originating from the CONTRACTOR CONSTRUCTION EQUIPMENT with a limit of not less than USD 10,000,000 per occurrence, noting that the two insurance coverage requirements named in Articles 41.5(a) and 41.5(b) may be addressed in one insurance policy.\n41.6 The CONTRACTOR shall ensure that all insurance policies provided by the CONTRACTOR shall: be obtained on an additional insured basis in the names of the PARTIES with the exception of the insurances required under Articles 41.1 and 41.2; and contain a waiver of subrogation against its contractual principals which shall include the COMPANY INDEMNIFIED PARTIES, to the extent of the liabilities assumed by the CONTRACTOR under the AGREEMENT.\n\n\n-----------------\n\n\n41.3 In the event the CONTRACTOR uses motor vehicles in the performance of the WORKS in the UAE, the CONTRACTOR shall obtain and maintain Motor Vehicle Third Party and Passenger Liability Insurance which shall be: as per the applicable LAWS of the UAE for death or injury to any THIRD PARTY; and for damage to PROPERTY of a THIRD PARTY with a limit of not less than USD 1,000,000 per occurrence.\n41.4 Without limitation to its obligations and responsibilities, the CONTRACTOR shall obtain and maintain All Risk Insurance on CONTRACTOR CONSTRUCTION EQUIPMENT covering the full value thereof.\n\n\n-----------------\n\n\nFor the avoidance of doubt, the governing law and dispute resolution provisions of Article 46 of ANNEXURE 2 \n \nPERFORMANCE BANK \nGUARANTEE amount: \n10% of the INITIAL AGREEMENT PRICE.] \n \nPARENT COMPANY \nGUARANTEE: \nAs Applicable \n \nAdditional CONTRACTOR \ninsurance requirements (in \naddition to those stated in \nArticles 41.1 to 41.15): \nNO  \n  \n \n \nTotal liability cap: \n100% of the AGREEMENT PRICE. \n \nREPRESENTATIVES: \nCOMPANY REPRESENTATIVE, as defined in Article 2.1(b), \nshall mean:  \nName:  \nTitle:  \nTel.: +971 \nEmail:  \nFax: [●]  \n \nCONTRACTOR REPRESENTATIVE, as defined in \nArticle 2.1(a), shall mean: \nName: [●] \nTitle: [●] \nTel.: [●] \nEmail: [●] \nFax: [●]  \n \nNotice details: \nNotices to the COMPANY: \nAttention:   \nAddress:  \nTel:  +971  \nEmail:   \nFax:  [insert] \n \nNotices to the CONTRACTOR: \nAttention: [insert name of CONTRACTOR REPRESENTATIVE] \nAddress: [insert] \nTel:  [insert] \nEmail: [insert] \nFax:  [insert]\n\n\n-----------------\n\n\nArticles 42.1(b), 42.2(b), 42.3(c) and 42.4(b).Notwithstanding Articles 42.9(a) and 42.9(b) above, no PARTY shall be deemed to have waived any claims against the other PARTY or its related INDEMNIFIED PARTIES in respect of events arising out of or in connection with the GROSS NEGLIGENCE / WILFUL MISCONDUCT of that other PARTY or its related INDEMNIFIED PARTIES. The COMPANY INDEMNIFIED PARTIES and the CONTRACTOR INDEMNIFIED PARTIES shall remain liable in respect of events arising out of or in connection with their GROSS NEGLIGENCE / WILFUL MISCONDUCT.\n42.10 LIMITATIONS OF LIABILITY \nThe total liability of either PARTY to the other PARTY arising under or in connection with the AGREEMENT shall be limited to the total liability cap set out in the FORM OF AGREEMENT. Article 42.10(a) shall not apply in relation to, or otherwise limit or affect:\n\n\n-----------------\n\n\n42. LIABILITY AND INDEMNITY\n42.1LIABILITY FOR PERSONNEL \nThe CONTRACTOR shall indemnify and hold harmless the COMPANY INDEMNIFIED PARTIES from and against all losses, liabilities, damages and costs (including legal costs) arising out of or in connection with any claim by any PERSON against the COMPANY INDEMNIFIED PARTIES in respect of death, illness of or injury to the CONTRACTOR PERSONNEL that arises out of or in connection with the performance or non-performance of the AGREEMENT by the COMPANY INDEMNIFIED PARTIES and/or the CONTRACTOR INDEMNIFIED PARTIES, except to the extent arising out of or in connection with the GROSS NEGLIGENCE / WILFUL MISCONDUCT of the COMPANY INDEMNIFIED PARTIES.\n\n\n-----------------\n\n\nWhere Article 16.3(d)(v) applies, the CONTRACTOR shall be required to reimburse such \nportion of the AGREEMENT PRICE as the COMPANY deems sufficient to compensate it for \nany losses, costs or expenses that it has suffered or may suffer as a consequence of, or in \nconnection with, the FACILITIES not achieving the PERFORMANCE GUARANTEES, up to \nthe amount of the CONTRACTOR’s total liability cap as set out in the FORM OF \nAGREEMENT.\n16.4 SUPPLY OF FUEL / HYDROCARBONS / UTILITIES AND/OR FEEDSTOCK To the extent that the COMPANY is responsible pursuant to ANNEXURE 6 for supply of fuel / hydrocarbons / utilities and/or FEEDSTOCK:\n\n\n-----------------\n\n\n47.2\nIf the COMPANY deems (at its absolute discretion) that any BUSINESS VENTURE by the \nCONTRACTOR under this Article 47 is prejudicial to the COMPANY’s interest, the COMPANY shall \ninstruct the CONTRACTOR in writing to take steps to terminate such BUSINESS VENTURE, and the \nCONTRACTOR shall comply with the COMPANY’s instruction. \n48. \nENTIRE AGREEMENT \nThe AGREEMENT constitutes the entire agreement between the PARTIES concerning the subject \nmatter of the AGREEMENT and supersedes and, subject to Article 3.2, replaces any previous \nagreements between the PARTIES or any representation made by one PARTY to the other PARTY \n(whether oral or written) concerning the subject matter of the AGREEMENT.  To the extent permitted \nby LAW, the PARTIES shall not be bound by or be liable for any statement, representation, promise, \ninducement or understanding of any kind or nature not set forth herein, provided that nothing in this \nArticle 48 shall limit a PARTY’s liability for fraud.\n\n\n-----------------\n\n\n42.2  LIABILITY FOR PROPERTY \n \nThe CONTRACTOR shall indemnify and hold harmless the COMPANY INDEMNIFIED PARTIES from and against all losses, liabilities, damages and costs (including legal costs) arising out of or in connection with any claim by any PERSON against the COMPANY INDEMNIFIED PARTIES in respect of:\n(i) loss of or damage to the PROPERTY of the CONTRACTOR INDEMNIFIED  PARTIES, including the loss of use thereof, that arises out of or in connection with the performance or non-performance of the AGREEMENT by the COMPANY INDEMNIFIED PARTIES and/or the CONTRACTOR INDEMNIFIED PARTIES; and\n\n\n-----------------\n\n\n(i) either PARTY’s liability in respect of: its GROSS NEGLIGENCE / WILFUL MISCONDUCT or fraud;  a breach of Article 37;  THIRD PARTIES; or an indemnity obligation to the other PARTY under Articles 4.3(e), 20.6, 22.3, 41.12, 41.16(d) and 42.1 to 42.5 (as applicable); \n(ii) the CONTRACTOR’s liability: in respect of its warranty of title pursuant to Article 33; or to make payments, to the extent corresponding payments: \n(1) are received or receivable by the CONTRACTOR, or adjusted or \ndeclined by insurers, pursuant to the insurance policies required by \nArticle 41, or \n(2) would have been received pursuant to such policies (including via \nreimbursement from the COMPANY) had the CONTRACTOR \ncomplied with its obligations under the AGREEMENT to effect and \nmaintain insurance; or \n(iii) the COMPANY’s liability as a result of the payment of any part of the AGREEMENT PRICE made in consideration of the CONTRACTOR’s performance of its obligations under the AGREEMENT.\n\n\n-----------------\n\n\nArticle 42. The CONTRACTOR shall save, indemnify, defend, release and hold harmless the COMPANY INDEMNIFIED PARTIES from and against all losses, liabilities, damages and costs (including legal costs) arising out of or in connection with any claim by any PERSON against the \nCOMPANY INDEMNIFIED PARTIES in respect of any shortfall in insurance cover it is obliged to obtain under the AGREEMENT or any failure to effect or maintain such insurances specified by the AGREEMENT or out of any act or omission, which invalidates the said insurances or in the event that \nliability for any loss or damage be denied by any underwriter, in all or in part, because of breach of sai\----------------\n\n\n-----------------\n\n\n\n\n\n -------------------  \n\n\n <Question>: What is CONTRACTOR limit of liability and the liability cap?".
  "stream":false
}'

In [ ]:
curl http://localhost:11434/api/generate -d '{
  "model": "llama3:8b-instruct-q4_K_M",
  "options":{"num_ctx":8000,"temperature":0.0,"top_p":0.5,},"prompt": " <Data>: \n \n \n 41.1 In the event any CONTRACTOR PERSONNEL are performing WORKS in the UAE, without limitation to its obligations and responsibilities, the CONTRACTOR shall obtain and maintain Workmen’s Compensation Insurance (Employer’s Liability) with a limit of liability of not less than USD 1,000,000 per occurrence.\n41.2 In the event any CONTRACTOR PERSONNEL are performing WORKS outside the UAE, without limitation to its obligations and responsibilities, the CONTRACTOR shall obtain and maintain Workmen’s Compensation Insurance (Employer’s Liability) or insurance coverage of a comparable \nnature in accordance with the applicable LAW and/or the contracts of employment of the CONTRACTOR PERSONNEL.\n\n\n-----------------\n\n\n41.5 The CONTRACTOR shall obtain and maintain: Third Party Liability Insurance with a limit of not less than USD 10,000,000 per occurrence; and Pollution Liability Insurance to cover the risk arising from the performance of WORKS and any pollution emanating/originating from the CONTRACTOR CONSTRUCTION EQUIPMENT with a limit of not less than USD 10,000,000 per occurrence, noting that the two insurance coverage requirements named in Articles 41.5(a) and 41.5(b) may be addressed in one insurance policy.\n41.6 The CONTRACTOR shall ensure that all insurance policies provided by the CONTRACTOR shall: be obtained on an additional insured basis in the names of the PARTIES with the exception of the insurances required under Articles 41.1 and 41.2; and contain a waiver of subrogation against its contractual principals which shall include the COMPANY INDEMNIFIED PARTIES, to the extent of the liabilities assumed by the CONTRACTOR under the AGREEMENT.\n\n\n-----------------\n\n\n41.3 In the event the CONTRACTOR uses motor vehicles in the performance of the WORKS in the UAE, the CONTRACTOR shall obtain and maintain Motor Vehicle Third Party and Passenger Liability Insurance which shall be: as per the applicable LAWS of the UAE for death or injury to any THIRD PARTY; and for damage to PROPERTY of a THIRD PARTY with a limit of not less than USD 1,000,000 per occurrence.\n41.4 Without limitation to its obligations and responsibilities, the CONTRACTOR shall obtain and maintain All Risk Insurance on CONTRACTOR CONSTRUCTION EQUIPMENT covering the full value thereof.\n\n\n-----------------\n\n\nFor the avoidance of doubt, the governing law and dispute resolution provisions of Article 46 of ANNEXURE 2 \n \nPERFORMANCE BANK \nGUARANTEE amount: \n10% of the INITIAL AGREEMENT PRICE.] \n \nPARENT COMPANY \nGUARANTEE: \nAs Applicable \n \nAdditional CONTRACTOR \ninsurance requirements (in \naddition to those stated in \nArticles 41.1 to 41.15): \nNO  \n  \n \n \nTotal liability cap: \n100% of the AGREEMENT PRICE. \n \nREPRESENTATIVES: \nCOMPANY REPRESENTATIVE, as defined in Article 2.1(b), \nshall mean:  \nName:  \nTitle:  \nTel.: +971 \nEmail:  \nFax: [●]  \n \nCONTRACTOR REPRESENTATIVE, as defined in \nArticle 2.1(a), shall mean: \nName: [●] \nTitle: [●] \nTel.: [●] \nEmail: [●] \nFax: [●]  \n \nNotice details: \nNotices to the COMPANY: \nAttention:   \nAddress:  \nTel:  +971  \nEmail:   \nFax:  [insert] \n \nNotices to the CONTRACTOR: \nAttention: [insert name of CONTRACTOR REPRESENTATIVE] \nAddress: [insert] \nTel:  [insert] \nEmail: [insert] \nFax:  [insert]\n\n\n-----------------\n\n\nArticles 42.1(b), 42.2(b), 42.3(c) and 42.4(b).Notwithstanding Articles 42.9(a) and 42.9(b) above, no PARTY shall be deemed to have waived any claims against the other PARTY or its related INDEMNIFIED PARTIES in respect of events arising out of or in connection with the GROSS NEGLIGENCE / WILFUL MISCONDUCT of that other PARTY or its related INDEMNIFIED PARTIES. The COMPANY INDEMNIFIED PARTIES and the CONTRACTOR INDEMNIFIED PARTIES shall remain liable in respect of events arising out of or in connection with their GROSS NEGLIGENCE / WILFUL MISCONDUCT.\n42.10 LIMITATIONS OF LIABILITY \nThe total liability of either PARTY to the other PARTY arising under or in connection with the AGREEMENT shall be limited to the total liability cap set out in the FORM OF AGREEMENT. Article 42.10(a) shall not apply in relation to, or otherwise limit or affect:\n\n\n-----------------\n\n\n42. LIABILITY AND INDEMNITY\n42.1LIABILITY FOR PERSONNEL \nThe CONTRACTOR shall indemnify and hold harmless the COMPANY INDEMNIFIED PARTIES from and against all losses, liabilities, damages and costs (including legal costs) arising out of or in connection with any claim by any PERSON against the COMPANY INDEMNIFIED PARTIES in respect of death, illness of or injury to the CONTRACTOR PERSONNEL that arises out of or in connection with the performance or non-performance of the AGREEMENT by the COMPANY INDEMNIFIED PARTIES and/or the CONTRACTOR INDEMNIFIED PARTIES, except to the extent arising out of or in connection with the GROSS NEGLIGENCE / WILFUL MISCONDUCT of the COMPANY INDEMNIFIED PARTIES.\n\n\n-----------------\n\n\nWhere Article 16.3(d)(v) applies, the CONTRACTOR shall be required to reimburse such \nportion of the AGREEMENT PRICE as the COMPANY deems sufficient to compensate it for \nany losses, costs or expenses that it has suffered or may suffer as a consequence of, or in \nconnection with, the FACILITIES not achieving the PERFORMANCE GUARANTEES, up to \nthe amount of the CONTRACTOR’s total liability cap as set out in the FORM OF \nAGREEMENT.\n16.4 SUPPLY OF FUEL / HYDROCARBONS / UTILITIES AND/OR FEEDSTOCK To the extent that the COMPANY is responsible pursuant to ANNEXURE 6 for supply of fuel / hydrocarbons / utilities and/or FEEDSTOCK:\n\n\n-----------------\n\n\n47.2\nIf the COMPANY deems (at its absolute discretion) that any BUSINESS VENTURE by the \nCONTRACTOR under this Article 47 is prejudicial to the COMPANY’s interest, the COMPANY shall \ninstruct the CONTRACTOR in writing to take steps to terminate such BUSINESS VENTURE, and the \nCONTRACTOR shall comply with the COMPANY’s instruction. \n48. \nENTIRE AGREEMENT \nThe AGREEMENT constitutes the entire agreement between the PARTIES concerning the subject \nmatter of the AGREEMENT and supersedes and, subject to Article 3.2, replaces any previous \nagreements between the PARTIES or any representation made by one PARTY to the other PARTY \n(whether oral or written) concerning the subject matter of the AGREEMENT.  To the extent permitted \nby LAW, the PARTIES shall not be bound by or be liable for any statement, representation, promise, \ninducement or understanding of any kind or nature not set forth herein, provided that nothing in this \nArticle 48 shall limit a PARTY’s liability for fraud.\n\n\n-----------------\n\n\n42.2  LIABILITY FOR PROPERTY \n \nThe CONTRACTOR shall indemnify and hold harmless the COMPANY INDEMNIFIED PARTIES from and against all losses, liabilities, damages and costs (including legal costs) arising out of or in connection with any claim by any PERSON against the COMPANY INDEMNIFIED PARTIES in respect of:\n(i) loss of or damage to the PROPERTY of the CONTRACTOR INDEMNIFIED  PARTIES, including the loss of use thereof, that arises out of or in connection with the performance or non-performance of the AGREEMENT by the COMPANY INDEMNIFIED PARTIES and/or the CONTRACTOR INDEMNIFIED PARTIES; and\n\n\n-----------------\n\n\n(i) either PARTY’s liability in respect of: its GROSS NEGLIGENCE / WILFUL MISCONDUCT or fraud;  a breach of Article 37;  THIRD PARTIES; or an indemnity obligation to the other PARTY under Articles 4.3(e), 20.6, 22.3, 41.12, 41.16(d) and 42.1 to 42.5 (as applicable); \n(ii) the CONTRACTOR’s liability: in respect of its warranty of title pursuant to Article 33; or to make payments, to the extent corresponding payments: \n(1) are received or receivable by the CONTRACTOR, or adjusted or \ndeclined by insurers, pursuant to the insurance policies required by \nArticle 41, or \n(2) would have been received pursuant to such policies (including via \nreimbursement from the COMPANY) had the CONTRACTOR \ncomplied with its obligations under the AGREEMENT to effect and \nmaintain insurance; or \n(iii) the COMPANY’s liability as a result of the payment of any part of the AGREEMENT PRICE made in consideration of the CONTRACTOR’s performance of its obligations under the AGREEMENT.\n\n\n-----------------\n\n\nArticle 42. The CONTRACTOR shall save, indemnify, defend, release and hold harmless the COMPANY INDEMNIFIED PARTIES from and against all losses, liabilities, damages and costs (including legal costs) arising out of or in connection with any claim by any PERSON against the \nCOMPANY INDEMNIFIED PARTIES in respect of any shortfall in insurance cover it is obliged to obtain under the AGREEMENT or any failure to effect or maintain such insurances specified by the AGREEMENT or out of any act or omission, which invalidates the said insurances or in the event that \nliability for any loss or damage be denied by any underwriter, in all or in part, because of breach of sai\----------------\n\n\n-----------------\n\n\n\n\n\n -------------------  \n\n\n <Question>: What is CONTRACTOR limit of liability and the liability cap?",  "stream": false
}'




top p=0.7

According to Article 41.5(a) of the agreement, the CONTRACTOR's limit of liability for Third Party Liability Insurance is USD 10,000,000 per occurrence.\n\nAdditionally, according to Article 42.10, the total liability of either PARTY (i.e., the COMPANY or the CONTRACTOR) to the other PARTY arising under or in connection with the AGREEMENT shall be limited to the total liability cap set out in the FORM OF AGREEMENT, which is not specified in this text.\n\nHowever, it can be inferred from Article 42.11(a) that the maximum amount of consequential loss that the COMPANY INDEMNIFIED PARTIES may recover from the CONTRACTOR INDEMNIFIED PARTIES is USD 1,000,000 per occurrence

default 

According to Article 41.5, the CONTRACTOR's Third Party Liability Insurance has a limit of not less than USD 10,000,000 per occurrence.\n\nAdditionally, according to Article 42.10(a), the total liability of either PARTY to the other PARTY arising under or in connection with the AGREEMENT shall be limited to the total liability cap set out in the FORM OF AGREEMENT.\n\nThe total liability cap is 100% of the AGREEMENT PRICE."

temperature=0   

According to Article 41.5, the CONTRACTOR's limit of liability for Third Party Liability Insurance and Pollution Liability Insurance is USD 10,000,000 per occurrence.\n\nAdditionally, according to Article 42.10(a), the total liability of either PARTY to the other PARTY arising under or in connection with the AGREEMENT shall be limited to the total liability cap set out in the FORM OF AGREEMENT, which is not specified in this text.

temperature =0 top_p=0.7

Based on the provided text, it appears that there are several instances where the CONTRACTOR's liability is limited or capped. Here are some specific examples:\n\n1. Article 42.3: The CONTRACTOR shall indemnify the COMPANY INDEMNIFIED PARTIES up to a maximum of USD 1,000,000 per occurrence for losses arising out of or in connection with any claim by any PERSON against the COMPANY INDEMNIFIED PARTIES.\n2. Article 42.12 (ii): The COMPANY shall indemnify the CONTRACTOR INDEMNIFIED PARTIES up to a maximum of USD 1,000,000 per occurrence for losses arising out of or in connection with any claim by any PERSON against the CONTRACTOR INDEMNIFIED PARTIES.\n\nIt appears that the liability cap is USD 1,000,000 per occurrence. However, it's worth noting that there may be other instances where the liability is limited or capped, and these examples are not exhaustive.", 


temperature = 0  top_p=0.5

"According to Article 41.5, the CONTRACTOR's limit of liability for Third Party Liability Insurance and Pollution Liability Insurance is USD 10,000,000 per occurrence.\n\nAdditionally, according to Article 42.10(a), the total liability of either PARTY to the other PARTY arising under or in connection with the AGREEMENT shall be limited to the total liability cap set out in the FORM OF AGREEMENT, which is not specified in this document. However, it can be inferred that the total liability cap is 100% of the AGREEMENT PRICE, as mentioned in Article 42.10(a)